---
title: "Matching Donations and Charitable Giving: A Replication Study"
author: "Ouwen Jia"
format: html
jupyter: python3
---

## Introduction

Charitable organizations often experiment with matching donation offers to encourage more contributions from potential donors. But does increasing the matching ratio always lead to more giving? This project replicates key findings from Karlan and List (2007), who conducted a large-scale field experiment using over 50,000 fundraising letters.

Our goal is to revisit their central question using the provided dataset and modern Python tools. We focus on both the overall treatment effect and explore whether different matching ratios (1:1, 2:1, 3:1) influence donor behavior differently.

## Data Overview

We use the dataset released alongside the study, which includes individual-level data on treatment assignment, prior donation behavior, demographic proxies, and outcomes of interest.

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_stata("karlan_list_2007.dta")

# Basic variable check
df[["treatment", "gave", "amount", "ratio", "size", "ask"]].head()

We focus on two main outcomes: whether a donation was made (`gave`), and the amount donated (`amount`). Treatment groups were assigned different match ratios and solicitation wordings.

## Exploratory Data Analysis

### Treatment vs. Control: Donation Rates

In [ ]:
import matplotlib.pyplot as plt

# Compare donation rates by treatment group
summary = df.groupby("treatment").agg(response_rate=("gave", "mean")).reset_index()
plt.bar(["Control", "Treatment"], summary["response_rate"], color=["#6baed6", "#fd8d3c"])
plt.title("Donation Rate: Treatment vs Control")
plt.ylabel("Proportion Donating")
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.show()

The figure shows a higher donation rate among individuals in the treatment group who received a matching offer.

### Match Ratio Effects (1:1 vs. 2:1 vs. 3:1)

In [ ]:
import seaborn as sns

# Only among treated
treated = df[df['treatment'] == 1]
ratio_summary = treated.groupby("ratio").agg(response_rate=("gave", "mean")).reset_index()

sns.barplot(data=ratio_summary, x="ratio", y="response_rate", palette="Set2")
plt.title("Donation Rate by Match Ratio")
plt.ylabel("Donation Rate")
plt.xlabel("Match Ratio")
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.show()

While all match ratios outperform the control group, increasing the match beyond 1:1 offers diminishing returns.

## Regression Analysis

To quantify the treatment effect, we fit a logistic regression where the dependent variable is whether an individual donated.

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

model = smf.logit("gave ~ treatment + ratio2 + ratio3", data=df).fit()

In [ ]:
# Clean output
model.summary2().tables[1].round(3)

The results confirm that the matching treatment significantly increases the probability of donating, while higher match ratios (2:1, 3:1) do not show significant improvements compared to the baseline 1:1.

## Heterogeneous Effects

The original study notes differences based on political context. We replicate this by comparing red vs. blue states:

In [ ]:
df["is_red_state"] = df["red0"] > 0.5

df.groupby("is_red_state").agg(
    control_rate=("gave", lambda x: x[df["treatment"] == 0].mean()),
    treatment_rate=("gave", lambda x: x[df["treatment"] == 1].mean())
)

We find that individuals in red states responded more to the matching offer, consistent with the original study's heterogeneity findings.

## Conclusion

In this replication, we confirm that matching donations do significantly raise response rates. However, increasing the match ratio does not meaningfully improve donation behavior beyond the basic 1:1 match. These findings support the original study and offer important implications for fundraising strategies.